# Pytorch Lessons

## Import main libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

from torch.utils.data import Dataset


## Tensors

Creating tensors and some tensor attibutes:

In [2]:
# Create tensor from numpy array:

X = np.random.normal(0,1,(10,10))
X = torch.tensor(X, dtype=torch.float64)


print('Element of a tensor:')
print(X[0,0])
print('Numpy content')
print(X.numpy())
print('Device')
print(X.device)
print('Shape')
print(X.shape)


Element of a tensor:
tensor(-0.3210, dtype=torch.float64)
Numpy content
[[-3.20984841e-01  8.68230526e-01  1.69571487e+00  7.23955389e-04
   3.36589140e-01 -8.83360495e-01  1.32287358e+00 -2.60220435e+00
  -1.33808755e-01  9.67779373e-01]
 [ 1.14607866e+00 -1.54071085e-02 -3.05711900e+00  4.54577189e-01
  -5.63976211e-01 -6.68318696e-02  1.78632671e+00  7.98796518e-01
  -6.11990172e-01  2.91468530e-01]
 [ 1.21132031e+00  9.45054357e-01 -2.15669023e+00  1.50055839e+00
  -2.06740392e+00  1.95280175e+00 -1.25718760e+00 -1.23448567e+00
   3.50098188e-01  9.84948695e-01]
 [-8.15555341e-01  9.27113340e-01  3.10052272e-01  1.23870895e+00
  -1.94882224e+00  9.04650961e-01  1.84186024e+00  1.81202203e+00
  -8.69199444e-02 -2.02965402e-01]
 [-8.56140589e-02  4.19903002e-02 -1.67506840e+00  1.48131658e+00
   1.79499347e+00 -1.17870425e+00  2.78287339e-01 -2.66689860e-01
   5.80595437e-02 -1.97701066e+00]
 [ 4.89294977e-01 -7.73010833e-01  1.33729802e-01  1.01537817e+00
  -6.96437630e-01  3.215189

Operations on tensors:

In [3]:

a = torch.tensor(np.arange(1,10).reshape(3,3), dtype=torch.float64)
b = torch.tensor(np.arange(10,19).reshape(3,3), dtype=torch.float64)

print('Sum of two tensors:')
print(a+b)
print('Tensor multiplication:')
print(a @ b)
print('Tensor transpose:')
print((a @ b).T)
print('Tensor multiplication:')
print(torch.matmul(a,b))
print('Mean:')
print(torch.mean(a))


Sum of two tensors:
tensor([[11., 13., 15.],
        [17., 19., 21.],
        [23., 25., 27.]], dtype=torch.float64)
Tensor multiplication:
tensor([[ 84.,  90.,  96.],
        [201., 216., 231.],
        [318., 342., 366.]], dtype=torch.float64)
Tensor transpose:
tensor([[ 84., 201., 318.],
        [ 90., 216., 342.],
        [ 96., 231., 366.]], dtype=torch.float64)
Tensor multiplication:
tensor([[ 84.,  90.,  96.],
        [201., 216., 231.],
        [318., 342., 366.]], dtype=torch.float64)
Mean:
tensor(5., dtype=torch.float64)


Inline operators have a _ in the end:

In [4]:
print(a)
a.transpose_(0,1)
print(a)
a.add_(5.0)
print(a)



tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], dtype=torch.float64)
tensor([[1., 4., 7.],
        [2., 5., 8.],
        [3., 6., 9.]], dtype=torch.float64)
tensor([[ 6.,  9., 12.],
        [ 7., 10., 13.],
        [ 8., 11., 14.]], dtype=torch.float64)


Numpy and torch

In [5]:
# In CPU
a = np.arange(1,10).reshape(3,3)
print(a)
t = torch.from_numpy(a)
print(t)
a[0,0] = 4
print(t)

t_ = torch.ones(3,3)

print(t_)

n_ = t_.numpy()

n_[0,0] = 5.0

print(t_)




[[1 2 3]
 [4 5 6]
 [7 8 9]]
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]], dtype=torch.int32)
tensor([[4, 2, 3],
        [4, 5, 6],
        [7, 8, 9]], dtype=torch.int32)
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[5., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [6]:
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

## Datasets

To implement a custom dataset, we use simulated data.



In [7]:
num = 10000
ep = 0.1

x = np.random.uniform(-4*np.pi,4*np.pi,(num,2))

y = np.sin(np.sqrt(x[:,0]**2+x[:,1]**2))+ ep*np.random.normal(0,1,num)


np.save('../DATA/x.npy', x)
np.save('../DATA/y.npy', y)


In [8]:
class CustomDataset(Dataset):
    def __init__(self, x_file, y_file):
        # Class constructor:
        # Loads the dataset from files:
        self.x = torch.tensor(np.load(x_file), dtype=torch.float64)
        self.y = torch.tensor(np.load(y_file), dtype=torch.float64)
        self.len = self.x.shape[0]

    def __getitem__(self, index):
        # Retrieves an item from the dataset:
        # In very big daytasets, we should not load all data at once.

        return self.x[index], self.y[index]
        # TODO: convert to tensor 
        
    def __len__(self):
        # Returns the length of the dataset:
        return self.len

In [9]:
data = CustomDataset('../DATA/x.npy', '../DATA/y.npy')

data.__getitem__(0)

(tensor([-0.6333,  5.1755], dtype=torch.float64),
 tensor(-0.8434, dtype=torch.float64))

## Dataloaders

In [10]:
v = np.arange(1,10)

it = iter(v)

print(next(it))
print(next(it))


1
2


In [11]:
train_loader = DataLoader(data, batch_size=64, shuffle=True)

x_batch, y_batch = next(iter(train_loader))

print(x_batch)

print(y_batch)



tensor([[ -6.7311, -11.5765],
        [-10.5169,   4.1648],
        [ -6.3972,   6.7122],
        [ -5.4248,  -8.1511],
        [ -9.0625, -10.9561],
        [  5.7612, -10.9601],
        [  4.4158,  -6.0960],
        [ -3.3264,  -2.8247],
        [  0.7254,  12.0318],
        [ -1.8539,  -1.6790],
        [ -3.1678,  -6.5105],
        [ 12.3219,  -7.3169],
        [  0.3138,  -4.3331],
        [  5.3345,  -7.1468],
        [  4.0946,  12.0322],
        [ -0.7887,  -7.3680],
        [ -1.1501,  11.6483],
        [ 10.5947,  -3.1375],
        [  6.2599,  -0.4490],
        [  5.5620,  -4.5110],
        [  7.6231,  -3.6295],
        [  1.8677,   2.1292],
        [ 11.3650,   0.1732],
        [ -9.0614,   6.9214],
        [ -2.6314,  -4.2140],
        [  0.9515,  -0.4879],
        [  3.7065,   2.5193],
        [  6.3062,   9.2012],
        [  1.9532,   2.4848],
        [ 10.4019,  11.3704],
        [ 10.6115,   0.8548],
        [ -1.5488,   2.0761],
        [ -6.2376,  -8.8461],
        [ 

## Create a sequential NN

In [12]:
class MyNNTest(nn.Module):

    def __init__(self):
        # call the parent class constructor
        super().__init__()

        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

In [13]:
# If we wanted to use GPU, we should check if it is available:
# model = NeuralNetwork().to(device)  

# To use double precision:
model = MyNNTest().double()
print(model)

MyNNTest(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [14]:
model(x_batch)

tensor([[ 0.7313],
        [ 0.4378],
        [ 0.1765],
        [ 0.5588],
        [ 0.7775],
        [ 1.1073],
        [ 0.7208],
        [ 0.3034],
        [ 0.2777],
        [ 0.2098],
        [ 0.4122],
        [ 0.9344],
        [ 0.3474],
        [ 0.8318],
        [ 0.1866],
        [ 0.4400],
        [ 0.2770],
        [ 0.2955],
        [ 0.0129],
        [ 0.6220],
        [ 0.4723],
        [-0.0031],
        [-0.1300],
        [ 0.2899],
        [ 0.3045],
        [ 0.1086],
        [-0.0566],
        [ 0.0646],
        [ 0.0023],
        [ 0.0756],
        [-0.1810],
        [ 0.0535],
        [ 0.6177],
        [ 0.4258],
        [-0.0832],
        [ 0.3317],
        [ 0.4906],
        [ 0.4608],
        [ 0.8545],
        [ 0.0893],
        [ 0.0271],
        [ 0.0914],
        [ 0.4068],
        [ 0.7942],
        [ 0.9437],
        [ 0.2914],
        [ 0.4249],
        [ 0.0934],
        [ 0.3803],
        [-0.2090],
        [ 0.5110],
        [ 0.5873],
        [ 0.

Model parameters

In [15]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 2]) | Values : tensor([[-0.1375, -0.3483],
        [-0.4689,  0.2882]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.3899, -0.2300], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0316, -0.0199,  0.0041,  ...,  0.0375,  0.0383, -0.0408],
        [ 0.0094, -0.0289,  0.0221,  ...,  0.0335,  0.0338, -0.0210]],
       dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0317,  0.0288], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([1, 512]) | Values : tensor([[-4.3247e-03,  3.2663e-03,  4.3945e-02,  3.3910e-02, -2.4253e-02,
         -1.7908e-02, -6.9334e-03,  1.3841e-02,  3.0994e-02,  8.2276e-03,
         -1.8599e-02, -1.3894e-02, 

Another way to access model params

In [16]:
model.state_dict()['linear_relu_stack.0.weight']

tensor([[-0.1375, -0.3483],
        [-0.4689,  0.2882],
        [ 0.1755,  0.5304],
        ...,
        [-0.3554,  0.3871],
        [ 0.1830,  0.0357],
        [ 0.4002, -0.0649]], dtype=torch.float64)

## Auto diff with torch.autograd

In [17]:
x = torch.tensor(np.linspace(-4*np.pi,4*np.pi,100), dtype=torch.float64)
